In [1]:
from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.transform import linear_cmap

import math
import pandas as pd
import numpy as np


output_notebook()

Loading BokehJS ...

In [2]:
full_path = 'Japan storms all years.csv'
tracks = pd.read_csv(full_path, sep=';')
tracks = tracks[tracks['Time of analysis'] != 66666]
print(tracks.head())
#print(tracks.info())

            Name  Time of analysis  Indicator  Grade  Lat   Lon  \
0  19901017_5101          51021906          2      2  200  1385   
1  19901017_5101          51021912          2      2  200  1385   
2  19901017_5101          51021918          2      2  230  1421   
3  19901017_5101          51022000          2      9  250  1460   
4  19901017_5101          51022006          2      9  276  1506   

   Central Pressure  Max sustained wind speed   a   b   c   d Landfall  
0              1010                       NaN NaN NaN NaN NaN      NaN  
1              1010                       NaN NaN NaN NaN NaN      NaN  
2              1000                       NaN NaN NaN NaN NaN      NaN  
3               994                       NaN NaN NaN NaN NaN      NaN  
4               994                       NaN NaN NaN NaN NaN      NaN  


In [3]:
def merc(lat, lon, unit=0.1):
    lat =lat * 0.1
    lon = lon * 0.1
    r_major = 6378137.000
    x = r_major * np.deg2rad(lon)
    scale = x/lon
    y = 180.0/math.pi * np.log(np.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

scaled_lat,scaled_lon = merc(tracks.Lat.values, tracks.Lon.values)
tracks['scaled_lat'] = scaled_lat
tracks['scaled_lon'] = scaled_lon

In [4]:
TOOLTIPS = [
    ("Name", "@name"),
   # ("(lat,lon)", "(@lat, @lon)"),
    ("Central Pressure", "@pressure"),
    ("Category", "@category"),
    ("Max sustained wind speed in knot", "@max_wind_speed")
]
p = figure(x_range=(14000000, 16000000), y_range=(1500000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator",tools='pan,hover,wheel_zoom',tooltips=TOOLTIPS)
p.add_tile(CARTODBPOSITRON)

bokeh.models.renderers.TileRenderer(
    id='325c7dc6-09d0-4edb-aee1-4cd02cb48ccd',
    alpha=1.0,
    js_event_callbacks={},
    js_property_callbacks={},
    level='underlay',
    name=None,
    render_parents=True,
    subscribed_events=[],
    tags=[],
    tile_source=bokeh.models.tiles.WMTSTileSource(
        id='873f26e8-0cdb-42ef-a511-714f999e6d34',
        attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors,&copy; <a href="https://cartodb.com/attributions">CartoDB</a>',
        extra_url_vars={},
        initial_resolution=156543.03392804097,
        js_event_callbacks={},
        js_property_callbacks={},
        max_zoom=30,
        min_zoom=0,
        name=None,
        snap_to_zoom=False,
        subscribed_events=[],
        tags=[],
        tile_size=256,
        url='https://tiles.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png',
        wrap_around=True,
        x_origin_offset=20037508.34,
        y_origin_offset=20037508.34),
    visible=True,
    x_range_name='default',
    y_range_name='default')

In [5]:
mapper = linear_cmap(field_name='category', palette=Spectral6, 
                         low=min(tracks['Grade'].values),
                         high=max(tracks['Grade'].values))

In [6]:
paths = tracks.groupby('Name')
#print(paths.groups.keys())
#print(paths.get_group('JEBI'))

In [7]:
#storms_to_plot = paths.groups.keys()
storms_to_plot = ['JEBI_1821', 'TRAMI_1824', 'VERA_5915','SONGDA_418','MIREILLE_9119']

In [8]:
for storm in storms_to_plot:
    path = paths.get_group(storm)
    source = ColumnDataSource(
        data=dict(lon=path.scaled_lon.values, lat=path.scaled_lat.values, pressure=path['Central Pressure'].values,
                  name=path.Name.values, category=path.Grade.values, max_wind_speed=path['Max sustained wind speed'].values)
    )

    p.line(x="lat", y="lon", line_color="blue", line_alpha=0.8, source=source)
    p.circle(x="lat", y="lon", size=7, fill_color=mapper,color=mapper, fill_alpha=0.8, source=source)
    #fill_color="blue", size="category"
show(p)